In [1]:
using FFTW, Random, Test

In [2]:
include("MacroUtils.jl")
using .MacroUtils: cleanup

Test data

In [3]:
Random.seed!(1234)
sample_vector_input = rand(32);
sample_tuple_input = Tuple(sample_vector_input);

In [4]:
function test_fft(fft)
    @test FFTW.fft(sample_vector_input) ≈ fft(sample_tuple_input)
end

test_fft (generic function with 1 method)

$$\omega_n = e^{-2\pi i / n}

$$ b_l = \sum_{k=1}^{n} \omega_n^{(k-1)(l-1)} a_k$$

In [5]:
function dft(a)
    n = length(a)
    @assert (n >= 1 && ispow2(n))
    w = exp(-2 * pi * im / n)
    wk = [w^(k - 1) for k in 1:n]
    # @show wk
    b = [sum([wk[k]^(l - 1) * a[k] for k in 1:n]) for l in 1:n]
    return b
end

dft (generic function with 1 method)

In [6]:
dft((10, 20, 30, 40))

4-element Vector{ComplexF64}:
               100.0 + 0.0im
 -20.000000000000007 + 19.999999999999996im
               -20.0 - 9.797174393178823e-15im
 -19.999999999999982 - 20.00000000000001im

In [7]:
test_fft(dft)

Test Passed

In [8]:
function fft_rec(a)
    n = length(a)
    @assert ispow2(n)
    isone(n) && return a
    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]
    y1 = fft_rec(a1)
    y2 = fft_rec(a2)

    wk = [w^(k - 1) for k in 1:n2]
    z1 = [y1[k] + wk[k] * y2[k] for k in 1:n2]
    z2 = [y1[k] - wk[k] * y2[k] for k in 1:n2]
    return vcat(z1, z2)
end

fft_rec (generic function with 1 method)

In [9]:
fft_rec((10, 20, 30, 40))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [10]:
test_fft(fft_rec)

Test Passed

In [11]:
function fft_rec2(a)
    n = length(a)
    @assert ispow2(n)

    b = Vector{Complex{Float64}}(undef, n)

    if isone(n)
        b[1] = a[1]
        return b
    end

    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]
    y1 = fft_rec2(a1)
    y2 = fft_rec2(a2)
    for k in 1:n2
        t = w^(k - 1) * y2[k]
        b[k] = y1[k] + t
        b[k+n2] = y1[k] - t
    end

    return b
end

fft_rec2 (generic function with 1 method)

In [12]:
fft_rec2((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [13]:
test_fft(fft_rec2)

Test Passed

In [14]:
function fft_gen_impl1!(a, es, m)
    n = length(a)

    b = Symbol("y_", m[])
    m[] += 1
    push!(es, :($b = Vector{Complex{Float64}}(undef, $n)))

    if isone(n)
        push!(es, :($b[1] = $(a[1])))
        return b
    end

    n2 = n ÷ 2
    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]

    y1 = fft_gen_impl1!(a1, es, m)
    y2 = fft_gen_impl1!(a2, es, m)

    w = exp(-2 * pi * im / n)

    for k in 1:n2
        t = Symbol("t_", m[])
        m[] += 1
        append!(es, (
            :($t = $(w^(k - 1)) * $y2[$k]),
            :($b[$k] = $y1[$k] + $t),
            :($b[$(k + n2)] = $y1[$k] - $t)
        ))
    end
    return b
end

fft_gen_impl1! (generic function with 1 method)

In [15]:
function fft_gen_impl1(n)
    @assert (n >= 1 && ispow2(n))
    a = [:(a[$k]) for k in 1:n]
    es = Expr[]
    m = Ref(1)
    b = fft_gen_impl1!(a, es, m)

    quote
        $(es...)
        return $b
    end
end

fft_gen_impl1 (generic function with 1 method)

In [16]:
fft_gen_impl1(8) |> cleanup

quote
    y_1 = Vector{Complex{Float64}}(undef, 8)
    y_2 = Vector{Complex{Float64}}(undef, 4)
    y_3 = Vector{Complex{Float64}}(undef, 2)
    y_4 = Vector{Complex{Float64}}(undef, 1)
    y_4[1] = a[1]
    y_5 = Vector{Complex{Float64}}(undef, 1)
    y_5[1] = a[5]
    t_6 = (1.0 + 0.0im) * y_5[1]
    y_3[1] = y_4[1] + t_6
    y_3[2] = y_4[1] - t_6
    y_7 = Vector{Complex{Float64}}(undef, 2)
    y_8 = Vector{Complex{Float64}}(undef, 1)
    y_8[1] = a[3]
    y_9 = Vector{Complex{Float64}}(undef, 1)
    y_9[1] = a[7]
    t_10 = (1.0 + 0.0im) * y_9[1]
    y_7[1] = y_8[1] + t_10
    y_7[2] = y_8[1] - t_10
    t_11 = (1.0 + 0.0im) * y_7[1]
    y_2[1] = y_3[1] + t_11
    y_2[3] = y_3[1] - t_11
    t_12 = (6.123233995736766e-17 - 1.0im) * y_7[2]
    y_2[2] = y_3[2] + t_12
    y_2[4] = y_3[2] - t_12
    y_13 = Vector{Complex{Float64}}(undef, 4)
    y_14 = Vector{Complex{Float64}}(undef, 2)
    y_15 = Vector{Complex{Float64}}(undef, 1)
    y_15[1] = a[2]
    y_16 = Vector{Complex{Float64}}(un

In [17]:
@generated function fft_gen1(a::NTuple{N, Float64}) where {N}
    fft_gen_impl1(N)
end

fft_gen1 (generic function with 1 method)

In [18]:
fft_gen1((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [19]:
test_fft(fft_gen1)

Test Passed

In [20]:
function fft_gen_impl2!(a, es, m)
    n = length(a)

    b = [Symbol("y_", m[], "_", k) for k in 1:n]
    m[] += 1

    if isone(n)
        push!(es, :($(b[1]) = $(a[1])))
        return b
    end

    n2 = n ÷ 2
    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]

    y1 = fft_gen_impl2!(a1, es, m)
    y2 = fft_gen_impl2!(a2, es, m)

    w = exp(-2 * pi * im / n)

    for k in 1:n2
        t = Symbol("t_", m[])
        m[] += 1
        append!(es, (
            :($t = $(w^(k - 1)) * $(y2[k])),
            :($(b[k]) = $(y1[k]) + $t),
            :($(b[k+n2]) = $(y1[k]) - $t)
        ))
    end
    return b
end

fft_gen_impl2! (generic function with 1 method)

In [21]:
function fft_gen_impl2(n)
    @assert (n >= 1 && ispow2(n))
    a = [:(a[$k]) for k in 1:n]
    es = Expr[]
    m = Ref(1)

    b = fft_gen_impl2!(a, es, m)

    quote
        $(es...)
        return $(Expr(:vect, b...))
    end
end

fft_gen_impl2 (generic function with 1 method)

In [22]:
fft_gen_impl2(8) |> cleanup

quote
    y_4_1 = a[1]
    y_5_1 = a[5]
    t_6 = (1.0 + 0.0im) * y_5_1
    y_3_1 = y_4_1 + t_6
    y_3_2 = y_4_1 - t_6
    y_8_1 = a[3]
    y_9_1 = a[7]
    t_10 = (1.0 + 0.0im) * y_9_1
    y_7_1 = y_8_1 + t_10
    y_7_2 = y_8_1 - t_10
    t_11 = (1.0 + 0.0im) * y_7_1
    y_2_1 = y_3_1 + t_11
    y_2_3 = y_3_1 - t_11
    t_12 = (6.123233995736766e-17 - 1.0im) * y_7_2
    y_2_2 = y_3_2 + t_12
    y_2_4 = y_3_2 - t_12
    y_15_1 = a[2]
    y_16_1 = a[6]
    t_17 = (1.0 + 0.0im) * y_16_1
    y_14_1 = y_15_1 + t_17
    y_14_2 = y_15_1 - t_17
    y_19_1 = a[4]
    y_20_1 = a[8]
    t_21 = (1.0 + 0.0im) * y_20_1
    y_18_1 = y_19_1 + t_21
    y_18_2 = y_19_1 - t_21
    t_22 = (1.0 + 0.0im) * y_18_1
    y_13_1 = y_14_1 + t_22
    y_13_3 = y_14_1 - t_22
    t_23 = (6.123233995736766e-17 - 1.0im) * y_18_2
    y_13_2 = y_14_2 + t_23
    y_13_4 = y_14_2 - t_23
    t_24 = (1.0 + 0.0im) * y_13_1
    y_1_1 = y_2_1 + t_24
    y_1_5 = y_2_1 - t_24
    t_25 = (0.7071067811865476 - 0.7071067811865475im

In [23]:
@generated function fft_gen2(a::NTuple{N, Float64}) where {N}
    fft_gen_impl2(N)
end

fft_gen2 (generic function with 1 method)

In [24]:
fft_gen2((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [25]:
test_fft(fft_gen2)

Test Passed

In [26]:
wp1(n, l) = mod(l, n) == 0
wm1(n, l) = 4 * mod(n - l, n) == 2 * n
wpi(n, l) = 4 * mod(n - l, n) == n
wmi(n, l) = 4 * mod(n - l, n) == 3 * n

wmi (generic function with 1 method)

In [38]:
[
    [wp1(8, k) for k in 0:7]'
    [wm1(8, k) for k in 0:7]'
    [wpi(8, k) for k in 0:7]'
    [wmi(8, k) for k in 0:7]'
]


4×8 Matrix{Bool}:
 1  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0
 0  0  0  0  0  0  1  0
 0  0  1  0  0  0  0  0

In [28]:
function butterfly(n, k, w, b1, b2, y1, y2, t)
    if wp1(n, k - 1)
        :($b1 = $y1 + $y2),
        :($b2 = $y1 - $y2)
    elseif wm1(n, k - 1)
        :($b1 = $y1 - $y2),
        :($b2 = $y1 + $y2)
    elseif wpi(n, k - 1)
        :($t = im * $y2),
        :($b1 = $y1 + $t),
        :($b2 = $y1 - $t)
    elseif wmi(n, k - 1)
        :($t = -im * $y2),
        :($b1 = $y1 + $t),
        :($b2 = $y1 - $t)
    else
        :($t = $(w^(k - 1)) * $y2),
        :($b1 = $y1 + $t),
        :($b2 = $y1 - $t)
    end
end

butterfly (generic function with 1 method)

In [29]:
function fft_gen_impl3!(a, es, m)
    n = length(a)

    b = [Symbol("y_", m[], "_", k) for k in 1:n]
    m[] += 1

    if isone(n)
        push!(es, :($(b[1]) = $(a[1])))
        return b
    end

    n2 = n ÷ 2
    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]

    y1 = fft_gen_impl3!(a1, es, m)
    y2 = fft_gen_impl3!(a2, es, m)

    w = exp(-2 * pi * im / n)

    for k in 1:n2
        t = Symbol("t_", m[])
        m[] += 1
        append!(es, butterfly(n, k, w, b[k], b[k+n2], y1[k], y2[k], t))
    end
    return b
end

fft_gen_impl3! (generic function with 1 method)

In [30]:
function fft_gen_impl3(n)
    @assert (n >= 1 && ispow2(n))
    a = [:(a[$k]) for k in 1:n]
    es = Expr[]
    m = Ref(1)

    b = fft_gen_impl3!(a, es, m)

    quote
        $(es...)
        return $(Expr(:vect, b...))
    end
end

fft_gen_impl3 (generic function with 1 method)

In [31]:
fft_gen_impl3(8) |> cleanup

quote
    y_4_1 = a[1]
    y_5_1 = a[5]
    y_3_1 = y_4_1 + y_5_1
    y_3_2 = y_4_1 - y_5_1
    y_8_1 = a[3]
    y_9_1 = a[7]
    y_7_1 = y_8_1 + y_9_1
    y_7_2 = y_8_1 - y_9_1
    y_2_1 = y_3_1 + y_7_1
    y_2_3 = y_3_1 - y_7_1
    t_12 = -im * y_7_2
    y_2_2 = y_3_2 + t_12
    y_2_4 = y_3_2 - t_12
    y_15_1 = a[2]
    y_16_1 = a[6]
    y_14_1 = y_15_1 + y_16_1
    y_14_2 = y_15_1 - y_16_1
    y_19_1 = a[4]
    y_20_1 = a[8]
    y_18_1 = y_19_1 + y_20_1
    y_18_2 = y_19_1 - y_20_1
    y_13_1 = y_14_1 + y_18_1
    y_13_3 = y_14_1 - y_18_1
    t_23 = -im * y_18_2
    y_13_2 = y_14_2 + t_23
    y_13_4 = y_14_2 - t_23
    y_1_1 = y_2_1 + y_13_1
    y_1_5 = y_2_1 - y_13_1
    t_25 = (0.7071067811865476 - 0.7071067811865475im) * y_13_2
    y_1_2 = y_2_2 + t_25
    y_1_6 = y_2_2 - t_25
    t_26 = -im * y_13_3
    y_1_3 = y_2_3 + t_26
    y_1_7 = y_2_3 - t_26
    t_27 = (-0.7071067811865474 - 0.7071067811865477im) * y_13_4
    y_1_4 = y_2_4 + t_27
    y_1_8 = y_2_4 - t_27
    return [y_1_

In [32]:
@generated function fft_gen3(a::NTuple{N, Float64}) where {N}
    fft_gen_impl3(N)
end

fft_gen3 (generic function with 1 method)

In [33]:
fft_gen3((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [34]:
test_fft(fft_gen3)

Test Passed